# Documentation Neuronal Network



In [ ]:

# We will use this cell to import all the packages you will need in the following - think of it as turning on all your systems
# in your cockpit

# This makes sure that if you change code in your external scripts, they will be updated

import numpy as np
from tensorflow import keras
import tensorflow as tf
import os
import datetime
import umap

cwd = os.getcwd ()

# now go ahead and Run the cell. This might take a while...
# while the cell is running, you will see ln[*] next to it. Once it finished, you will see the number of execution
# In case you want to interrupt the run of a cell, press Ctrl + C (on your german keyboard, this is Strg + C) 


## Loading the MNIST Dataset
To build and train our model, we must load the MNIST dataset. 
Once the data has been loaded, we will print both the training and validation datasets.

In [ ]:
# Loading the MNIST dataset in one line
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Printing the shape
print('x_train:', x_train.shape)
print('y_train:', y_train.shape)
print('x_test:', x_test.shape)
print('y_test:', y_test.shape)

## Defining our function to calculate the mse of two pictures

This function calculates the mean squared error (MSE) between two labels to determine their similarity and differences. It is used to identify the optimal input labels for our neural network.


In [4]:
def calculate_mse(imageA, imageB):
	err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
	err /= float(imageA.shape[0] * imageA.shape[1])
	
	return err

# Defining the function to search for the most similar and best images.

The primary function is to obtain the top 10 labels from our dataset. To achieve this, we calculate the mean squared error of a specific label with all other labels of that number. This algorithm enables us to identify the most suitable representative label for each number, which we can then use for training.

The second function calculates the mean squared error (MSE) of one label compared to all other labels with the same number. The aim is to identify the most similar input to our first label and then remove it to obtain the best mix of labels in our dataset.

We will use only the top 10 labels from the MNIST dataset to train our model. Calculating the labels is time-consuming as we need to calculate the mean squared error of each label with every other label of that number. Therefore, we have already calculated the top 10 labels and will define them in an array to save time.

In [ ]:
'''keep_numbers=[]
mse_numbers = []
def searchBestImage():
  for number in range (0,10):
    mse_number = 0.0
    filter_for_number = np.where(y_train == number)[0]
    for number1 in range (len(filter_for_number)):
      for number2 in range (len(filter_for_number)):
        mse_number += calculate_mse(x_train[filter_for_number[number1]],x_train[filter_for_number[number2]])
      mse_number = mse_number / len(filter_for_number)
      mse_numbers.append(mse_number)
    min = np.argmin(mse_numbers)
    keep_index.append(filter_for_number[min])
    mse_numbers.clear()
    filter_for_number = np.where(y_train == number )[0]


x_train, y_train = x_train[0:5000], y_train[0:5000] # This is just if we want to take more labels and search the similar labels in a dataset to remove them

def search_for_most_similar_input(list, index):
  store_mse = 1.0
  remember_index = 1
  for i in range (index, len(list)):
    if i != index: 
      actual_mse = calculate_mse(list[index], list[i])
      if actual_mse < store_mse:
        store_mse = actual_mse
        remember_index = i
  return remember_index
for calc in range (0,5):
  for i in range (0,10):
    filter_for_number = np.where(y_train == i)[0]
    for y in range (len(x_train[filter_for_number])//2):
      index = search_for_most_similar_input(x_train[filter_for_number], y)
      x_train = np.delete(x_train, filter_for_number[index], 0)
      y_train = np.delete(y_train,filter_for_number[index])
      filter_for_number = np.where(y_train == i )[0]
'''
keep_index = [39306, 45926, 35779, 9059, 3639, 29049, 15459, 26151, 56284, 1733]

In [ ]:
x_train = x_train/255
x_test = x_test/255

x_train= x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# Defining UMAP Model

We first try to load the reduced dataset from a file to save time transforming the data. 

# Reshaping the training data

To begin defining and training our model, we must first prepare our training dataset. 
This involves reshaping the training and validation data to fit our input layer. 
Additionally, we must reshape our input to fit the input layer later on. 
This is because Keras expects a definition for the number of channels in the input sample. 
As we are dealing with grayscale, this value is 1.

To improve the accuracy of our model and reduce the dimensionality of our input data, we utilise a UMAP model to transform our training data.



In [ ]:

try: 
   x_train_reduced=np.load('training_data/x_train_reduced.npy')
   x_test_reduced=np.load('training_data/x_test_reduced.npy')
   print("Filtered data for training were loaded!")
except:
  x_train = x_train/255
  x_test = x_test/255
  x_train= x_train.reshape(-1, 784)
  x_test = x_test.reshape(-1, 784)
  print('x_train:', x_train.shape)
  print('y_train:', y_train.shape)
  print('x_test:', x_test.shape)
  print('y_test:', y_test.shape)
  n_components=10
  umap_model = umap.UMAP(n_components=n_components,)
  x_train_reduced = umap_model.fit_transform(x_train)
  x_test_reduced = umap_model.transform(x_test)
  np.save('training_data/x_train_reduced.npy',x_train_reduced)
  np.save('training_data/x_test_reduced.npy',x_test_reduced)

# Print out the training data 

Before starting training, it is necessary to examine the training data and the shape of the dataset after dimension reduction.

In [ ]:
x_train_reduced=x_train_reduced[keep_index]
y_train=y_train[keep_index]
print("y_train:",y_train)
print('x_red:', x_train_reduced.shape)

# Defining our Model

Now we can define our model.

In [ ]:
marvin = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(n_components,)),
  tf.keras.layers.Dense(128, activation='sigmoid',kernel_initializer='glorot_uniform'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(10, activation='softmax',kernel_initializer='glorot_uniform', use_bias=True)
])
marvin.summary()

# Setting model parameters

Before starting model training, it is necessary to set certain model parameters. These parameters include the optimizer to be used, the loss function, the metrics, and the reduction of learning rate and model checkpoints.

In [19]:
marvin.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
              loss= 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau (monitor ='val_accuracy', factor =0.5 , min_delta=0.00005 , 
                                                  patience=6, min_lr=0.0001, mode="max")
model_checkpoint = tf.keras.callbacks.ModelCheckpoint (filepath=os.path.join(cwd,'models/model_1.h5'),
                                                       monitor='val_accuracy',save_best_only=True, mode="max")

# Fit Model

The model is trained using the defined model parameters and the training dataset. The training process consists of 200 epochs with a batch size of 32.

In [ ]:
marvin.fit(
    x_train_reduced,
    y_train,
    epochs= 200,
    batch_size= 32,
    validation_data=(x_test_reduced, y_test),
    callbacks=[model_checkpoint,reduce_lr]
)

# Safe Model

To use our model later and reload it, we now want to save it.

In [ ]:
#SAVE MODEL 
model_name = 'model_1'
marvin.save(model_name, save_format='h5')

print('Success! You saved Marvin as: ', model_name)

# Evaluate Model

Now, we need to reload and evaluate the model to confirm the validation accuracy.

In [ ]:
marvin_reloaded = tf.keras.models.load_model('models/model_1.h5')
marvin_reloaded.summary()
loss_and_metrics = marvin_reloaded.evaluate(x_test_reduced, y_test, verbose=2)

print("Test Loss", loss_and_metrics[0])
print("Test Accuracy", loss_and_metrics[1])

predictions = marvin_reloaded.predict([x_test_reduced])

predictions = np.argmax(predictions, axis=1)